In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
company_df = spark.read.table('silver.movie_company')
company_df.show()

+-------+---------+
|movieId|companyId|
+-------+---------+
|      1|       14|
|      1|       85|
|      3|     1679|
|      3|    13437|
|      4|      249|
|      4|    10906|
|      4|    10907|
|      5|    10324|
|      6|       60|
|      6|     2532|
|      6|    12187|
|      7|       60|
|      7|    11241|
|      8|     2342|
|      9|     4263|
|     10|      306|
|     11|     8411|
|     12|      220|
|     12|      221|
|     12|      222|
+-------+---------+
only showing top 20 rows



In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# Define the silver location
company_goldLocation = "/mnt/datalake/gold/dim_movie_company"

# Table name
company_goldTable = "gold.Dim_Movie_Company"

company_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(company_goldLocation)

In [0]:
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {company_goldTable} USING delta LOCATION '{company_goldLocation}'")

DataFrame[]

In [0]:
# Maintenance for Delta table
company_DataDelta = DeltaTable.forName(spark, company_goldTable)

if company_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	company_DataDelta.optimize()
	company_DataDelta.vacuum()


In [0]:
%sql 
select * from gold.dim_movie_company

movieId,companyId
1,14
1,85
3,1679
3,13437
4,249
4,10906
4,10907
5,10324
6,60
6,2532
